In [1]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

/Users/gordontveito-duncan/Documents/GitHub/prompt-engineering-for-generative-ai-examples/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Custom imports:
from content_collection import collect_serp_data_and_extract_text_from_webpages
from custom_summarize_chain import create_all_summaries, DocumentSummary
from expert_interview_chain import InterviewChain

In [3]:
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

True

## Topic Research:

%pip install google-search-results pandas html2text pytest-playwright chromadb nest_asyncio --quiet

!playwright install

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
# Constant variables:
TOPIC = "Neural networks"

In [6]:
# Extract content from webpages into LangChain documents:
text_documents = await \
    collect_serp_data_and_extract_text_from_webpages(topic=TOPIC)

In [7]:
# LLM, text splitter + parser:
llm = ChatOpenAI(temperature=0)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1500, chunk_overlap=400
)
parser = PydanticOutputParser(pydantic_object=DocumentSummary)

In [8]:
summaries = await create_all_summaries(text_documents, parser, llm, text_splitter)

/Users/gordontveito-duncan/Documents/GitHub/prompt-engineering-for-generative-ai-examples/content/chapter_10/custom_summarize_chain.py:51: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))
/Users/gordontveito-duncan/Documents/GitHub/prompt-engineering-for-generative-ai-examples/content/chapter_10/custom_summarize_chain.py:57: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  stuff_chain = StuffDocumentsChain(


Summarizing the data!
Summarizing the data!
Summarizing the data!
Time taken: 2.041219711303711
Finished summarizing the data!
---
Time taken: 3.7546639442443848
Finished summarizing the data!
---
Time taken: 4.385705947875977
Finished summarizing the data!
---


---

## Expert Interview Questions:


In [8]:
interview_chain = InterviewChain(topic=TOPIC, document_summaries=summaries)
interview_questions = interview_chain()

----------------------------------------

## Answer The Interview Questions:

In [ ]:
for question in interview_questions.questions:
    print(f"Answer the following question: {question.question}\n", flush=True)
    answer = input(f"Answer the following question: {question.question}\n")
    print('------------------------------------------')
    question.answer = answer

Answer the following question: Can you explain the significance of neural networks in the field of artificial intelligence and machine learning?

------------------------------------------
Answer the following question: What are some key differences between traditional machine learning algorithms and neural networks?

------------------------------------------
Answer the following question: How do neural networks mimic the way biological neurons work in the human brain?



KeyboardInterrupt: Interrupted by user

---

## General Article Outline:


In [ ]:
from article_outline_generation import BlogOutlineGenerator

blog_outline_generator = BlogOutlineGenerator(topic=TOPIC, questions_and_answers=[item.dict()  
                                                                                  for item in interview_questions.questions ] )
questions_and_answers = blog_outline_generator.questions_and_answers
outline_result = blog_outline_generator.generate_outline(summaries)

---

## Article Text Generation:


In [ ]:
from article_generation import ContentGenerator

content_gen = ContentGenerator(topic=TOPIC, outline=outline_result, questions_and_answers=questions_and_answers)

# Vectorize and store the original webpages:
content_gen.split_and_vectorize_documents(text_documents)

# Create the blog post (this may take several minutes to run, so please be patient):
blog_post = content_gen.generate_blog_post()

In [ ]:
blog_post

-----

## Image Creation:

In [ ]:
from image_generation_chain import create_image
image = create_image(outline_result.title)

In [ ]:
print(f"The image is ready! The filepath is {image[0]}")